In [39]:
from azureml.core import Workspace, Experiment, Run
import math, random, pickle, json
from azureml.core.model import Model

In [40]:
ws = Workspace.from_config()

Found the config file in: /home/nbuser/library/config.json


In [41]:
experiment_name = "biohack-experiment"
run = list(Experiment(workspace = ws, name = experiment_name).get_runs())[0]

In [42]:
model = run.register_model(model_name = "pi_estimate.pkl", model_path = "outputs/pi_estimate.pkl")

In [43]:
%%writefile score.py
import pickle, json
from azureml.core.model import Model
import numpy as np

def init():
    global pi_estimate
    model_path = Model.get_model_path(model_name = "pi_estimate.pkl")
    with open(model_path, "rb") as f:
        pi_estimate = pickle.load(f)

def run(raw_data):
    try:
        tmp = json.loads(raw_data)["tmp"]
        ad = json.loads(raw_data)["ad"]
        chss = json.loads(raw_data)["chss"]
        chd = json.loads(raw_data)["chd"]
        laktat = json.loads(raw_data)["laktat"]
        countLeiko = json.loads(raw_data)["countLeiko"]
        
        js = np.array([float(tmp), float(ad),float(chss),float(chd), float(laktat),float(countLeiko)])
        js = js.reshape(1,-1)
        result = pi_estimate.predict_proba(js)
        result = result.take(1)
        print(type(pi_estimate))
        return json.dumps({"result": result})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

Overwriting score.py


In [44]:
from azureml.core.conda_dependencies import CondaDependencies

cd = CondaDependencies()
cd.add_pip_package("scikit-learn==0.19.1")
cd.add_pip_package("numpy==1.16.0")
cd.add_pip_package("scipy")
cd.add_pip_package("matplotlib")

#cd.save_to_file(".", "myenv.yml")
with open("myenv.yml","w") as f:
    f.write(cd.serialize_to_string())

In [45]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage

# Define the configuration of compute: ACI with 1 cpu core and 1 gb of memory.
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

# Specify the configuration of image: scoring script, Python runtime (PySpark is the other option), and conda file of library dependencies.
image_config = ContainerImage.image_configuration(execution_script = "score.py",
runtime = "python",
conda_file = "myenv.yml")

# Deploy the web service as an image containing the registered model.
service = Webservice.deploy_from_model(name = "cors",deployment_config = aci_config,models = [model],image_config = image_config,workspace = ws)

# The service deployment can take several minutes: wait for completion.
service.wait_for_deployment(show_output = True)

Creating image
Image creation operation finished for image cors:1, operation "Succeeded"
Creating service
Running...............
SucceededACI service creation operation finished, operation "Succeeded"


In [ ]:
from ipywidgets import interact # for test

def get_area(radius):
    request = json.dumps({"radius": radius})# не правильно
    response = service.run(input_data = request) # не правильно
    return json.loads(response)["area"] # не правильно

interact(get_area,radius=(0,10))